In [1]:
import graphlab as gl
import datetime as dt

[WARNING] graphlab.product_key: Unable to write current GraphLab Create license to /home/nawel/.graphlab/config. Ensure that this user account                         has write permission to /home/nawel/.graphlab/config to save the license for offline use.
[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1482941561.log


This non-commercial license of GraphLab Create for academic use is assigned to nawel.medjkoune@u-psud.fr and will expire on December 27, 2017.


[WARNING] graphlab.deploy._session: Unable to create session in specified location: '/home/nawel/.graphlab/artifacts'. Using: '/var/tmp/graphlab-nawel/13792/tmp_session_75e88ebb-0f3e-4746-be03-704521e4222f'


# Loading data

In [2]:
data=gl.SFrame.read_csv('../../../train_2011_2012_2013.csv', delimiter=';', verbose=False)


In [ ]:
data[''].unique()

# Feature Engineering

In [6]:
#date splitting
date_col=data['DATE'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.000'))
splitted_date_col=date_col.split_datetime(column_name_prefix='', limit=['year', 'month', 'day', 'hour', 'minute'])
for col in ['year', 'month', 'day', 'hour', 'minute']:
    data[col] = splitted_date_col[col]
data['DAY_WE_DS'] = date_col.apply(lambda x: x.weekday())

In [ ]:
#One hot encoder
#from graphlab.toolkits.feature_engineering import OneHotEncoder
#encoder = gl.feature_engineering.create(data, OneHotEncoder(features = ['ASS_ASSIGNMENT'], output_column_name='ASS_ASSIGNMENT'))
#encoder = gl.feature_engineering.create(data, OneHotEncoder(features = ['TPER_TEAM','ASS_DIRECTORSHIP']))


In [ ]:
#transformed_data = encoder.transform(data)

# Time series (no use)

In [ ]:
#2011-04-24 01:30:00.000
#data['DATE'] = data['DATE'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.000'))

In [ ]:
#data_ts = gl.TimeSeries(data, index="DATE")


In [ ]:
#start = dt.datetime(2011,1,1)
#end  = dt.datetime(2012,1,1)
#ts_2011 = data_ts.slice(start, end)

# Load test set

In [8]:
test=gl.SFrame.read_csv('../../../submission.txt', delimiter='\t', verbose=False)
test=test['DATE' , 'ASS_ASSIGNMENT']

In [9]:
# feature engineering
date_col=test['DATE'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.000'))
splitted_date_col=date_col.split_datetime(column_name_prefix='', limit=['year', 'month', 'day', 'hour', 'minute'])
for col in ['year', 'month', 'day', 'hour', 'minute']:
    test[col] = splitted_date_col[col]
test['DAY_WE_DS'] = date_col.apply(lambda x: x.weekday())

In [ ]:
#transformed_test = encoder.transform(test)

# Model prediction

In [ ]:
model = gl.boosted_trees_regression.create(data, target='CSPL_RECEIVED_CALLS', features=['year', 'month', 'day', 'hour', 'WEEK_END', 'DAY_WE_DS', 'ASS_ASSIGNMENT'], max_iterations=2, max_depth =  3)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



In [ ]:
predictions=model.predict(test)
#predictions=predictions.apply(lambda x: int(x))

In [ ]:
def make_submission(prediction, filename='submission.txt'):
    with open(filename, 'w') as f:
        f.write('DATE\tASS_ASSIGNMENT\tprediction\n')
        submission_strings = test['DATE'] + '\t' + test['ASS_ASSIGNMENT'] + '\t'+ prediction.astype(str)
        for row in submission_strings:
            f.write(row + '\n')

make_submission(predictions, 'submission.txt')


